# Load Example Data Into the Exasol database

In this Notebook we will load the "Air pressure system failures in Scania trucks" dataset into the exasol database. It is a predictive maintenance scenario:

> The dataset consists of data collected from heavy Scania trucks in everyday usage. The system in focus is the Air Pressure system (APS) which generates pressurized air that is utilized in various functions in a truck, such as braking and gear changes. The datasets' positive class consists of component failures for a specific component of the APS system. The negative class consists of trucks with failures for components not related to the APS. The data consists of a subset of all available data, selected by experts. 

In [ ]:
EXASOL_HOST = "<database_host>" # change
EXASOL_PORT = "8563" # change if needed
EXASOL_USER = "sys" # change if needed
EXASOL_PASSWORD = "<database_password>" #change
EXASOL_SCHEMA = "IDA"

In [ ]:
!pip install pyexasol

import pyexasol
from io import BytesIO
from urllib.request import urlopen
import pandas as pd
from zipfile import ZipFile

In [ ]:
EXASOL_CONNECTION = "{host}:{port}".format(host=EXASOL_HOST, port=EXASOL_PORT)
exasol = pyexasol.connect(dsn=EXASOL_CONNECTION, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)

## Download Example Data

In [ ]:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00414/to_uci.zip"
TRAINING_FILE = "to_uci/aps_failure_training_set.csv"
TEST_FILE = "to_uci/aps_failure_test_set.csv"

# Data is preceeded with a 20-line header (copyright & license)
NUM_SKIP_ROWS = 20
NA_VALUE = "na"

resp = urlopen(DATA_URL)
with open('to_uci.zip', 'wb') as f:  
    f.write(resp.read())
    
print("data downloaded")

In [ ]:
with ZipFile('to_uci.zip') as z:
    with z.open(TRAINING_FILE, "r") as f:
        train_set = pd.read_csv(f, skiprows=NUM_SKIP_ROWS, na_values=NA_VALUE)
    with z.open(TEST_FILE, "r") as f:
        test_set = pd.read_csv(f, skiprows=NUM_SKIP_ROWS, na_values=NA_VALUE)

## Import Example Data

In [ ]:
exasol.execute(query="CREATE SCHEMA IF NOT EXISTS {schema!i}", query_params={"schema": EXASOL_SCHEMA})

In [ ]:
# Define column names and types
column_names = list(train_set.columns)
column_types = ["VARCHAR(3)"] + ["DECIMAL(18,2)"] * (len(column_names) - 1)
column_desc = [" ".join(t) for t in zip(column_names, column_types)]

params = {"schema": EXASOL_SCHEMA, "column_names": column_names, "column_desc": column_desc}

# Create tables for data
exasol.execute(query="CREATE OR REPLACE TABLE {schema!i}.TRAIN(" + ", ".join(column_desc) + ")", query_params=params)
exasol.execute(query="CREATE OR REPLACE TABLE {schema!i}.TEST LIKE {schema!i}.TRAIN", query_params=params)

# Import data into Exasol
exasol.import_from_pandas(train_set, (EXASOL_SCHEMA, "TRAIN"))
print(f"Imported {exasol.last_statement().rowcount()} rows into TRAIN.")
exasol.import_from_pandas(test_set, (EXASOL_SCHEMA, "TEST"))
print(f"Imported {exasol.last_statement().rowcount()} rows into TEST.")